In [1]:
import json

from langchain_core.documents import Document

def f1(fname): # load json to documents in llm - dataloading
    with open(fname) as fobj:
        data = json.load(fobj)

    docs = []
    for var in data:
        content = var.get("content"," ")
        metadata_result = {k: v for k,v, in var.items() if k != 'content'}
        docs.append(Document(page_content=content,metadata=metadata_result))
    return docs
        

In [2]:
data = f1('data.json')
data

[Document(metadata={'title': 'LangChain Overview'}, page_content='LangChain is a framework for developing applications powered by language models.'),
 Document(metadata={'title': 'What is Ollama?'}, page_content='Ollama allows running large language models locally like LLaMA and Mistral.'),
 Document(metadata={'title': 'Embeddings in NLP'}, page_content='Embeddings are vector representations of text used for similarity and retrieval.')]

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma

In [4]:
splitter = RecursiveCharacterTextSplitter(chunk_size=300,chunk_overlap=50)
splitted_docs = splitter.split_documents(data)

embedding = OllamaEmbeddings(model="gemma:2b")

vectordb = Chroma.from_documents(splitted_docs,embedding=embedding)
retriver = vectordb.as_retriever()

C:\Users\karth\AppData\Local\Temp\ipykernel_20172\933803318.py:4: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding = OllamaEmbeddings(model="gemma:2b")


In [5]:
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain 
from langchain.chains import create_retrieval_chain
from langchain_community.llms import Ollama

In [7]:
# define your prompt
prompt_obj = PromptTemplate.from_template('''You are an expert assistant {context} Question:{input} Answer:''')
# connect llm
llm_obj = Ollama(model='gemma:2b')

# create stuff chain to combine retrived docs
doc_chain = create_stuff_documents_chain(llm=llm_obj,prompt=prompt_obj)
# create retrieved chain
rag_chain = create_retrieval_chain(retriever=retriver,combine_docs_chain=doc_chain)

# invoke query
response = rag_chain.invoke({'input':'what is langchain?'})
# display response
print(response)

{'input': 'what is langchain?', 'context': [Document(metadata={'title': 'LangChain Overview'}, page_content='LangChain is a framework for developing applications powered by language models.'), Document(metadata={'title': 'What is Ollama?'}, page_content='Ollama allows running large language models locally like LLaMA and Mistral.'), Document(metadata={'title': 'Embeddings in NLP'}, page_content='Embeddings are vector representations of text used for similarity and retrieval.')], 'answer': "Sure, here's a summary of the answer:\n\n**LangChain** is a framework for developing applications powered by language models. It allows users to run large language models locally, enabling them to build and deploy various applications that leverage the capabilities of these models."}


In [8]:
import pprint
pprint.pprint(response)

{'answer': "Sure, here's a summary of the answer:\n"
           '\n'
           '**LangChain** is a framework for developing applications powered '
           'by language models. It allows users to run large language models '
           'locally, enabling them to build and deploy various applications '
           'that leverage the capabilities of these models.',
 'context': [Document(metadata={'title': 'LangChain Overview'}, page_content='LangChain is a framework for developing applications powered by language models.'),
             Document(metadata={'title': 'What is Ollama?'}, page_content='Ollama allows running large language models locally like LLaMA and Mistral.'),
             Document(metadata={'title': 'Embeddings in NLP'}, page_content='Embeddings are vector representations of text used for similarity and retrieval.')],
 'input': 'what is langchain?'}


In [9]:
pprint.pprint(response['answer'])

("Sure, here's a summary of the answer:\n"
 '\n'
 '**LangChain** is a framework for developing applications powered by language '
 'models. It allows users to run large language models locally, enabling them '
 'to build and deploy various applications that leverage the capabilities of '
 'these models.')


In [10]:
from langchain_core.prompts import ChatPromptTemplate
chat_prompt = ChatPromptTemplate.from_messages([('system','you are exper assistant. use the below context to answer the question'),
                                  ('human','content:\n{context}\n\nQuestion: {input}')])

llmobj = Ollama(model="gemma:2b")

doc_chain = create_stuff_documents_chain(llm=llmobj,prompt=chat_prompt)

rag_chain = create_retrieval_chain(retriever=retriver,combine_docs_chain=doc_chain)

# invoke query
response = rag_chain.invoke({'input':'what is langchain?'})
# display response
print(response)


{'input': 'what is langchain?', 'context': [Document(metadata={'title': 'LangChain Overview'}, page_content='LangChain is a framework for developing applications powered by language models.'), Document(metadata={'title': 'What is Ollama?'}, page_content='Ollama allows running large language models locally like LLaMA and Mistral.'), Document(metadata={'title': 'Embeddings in NLP'}, page_content='Embeddings are vector representations of text used for similarity and retrieval.')], 'answer': 'The context does not provide any information about what langchain is, so I cannot answer this question from the provided context.'}
